In [5]:
import pandas as pd
import numpy as np
import datetime
import gc
import random

In [6]:
product_login = pd.read_csv('./FE_data/product_login.csv')
users_info_all = pd.read_csv('./users_info_all.csv')

In [7]:
a = np.setdiff1d(product_login['role_id'], users_info_all['role_id'])
print(len(a))
b = np.setdiff1d(users_info_all['role_id'], product_login['role_id'])
print(len(b))

210928
3241


In [8]:
# 合并
data_3d = pd.merge(product_login, users_info_all, on=['user_id', 'role_id'], how='left', validate='one_to_one')
pd.set_option('display.max_columns', None)
data_3d.head()

,user_id,role_id,r1,r2,r3,r4,max_role_level,is_pay,p1,p2,p3,p4,p5,p6,is_create_role,is_create_no,create_role_time,server_sum,is_gunfu,platform,user_type,user_flag,mobile,channel_id,source_id,device_no,reg_date,game_pay_num,chmoney_pay_num
0,96918420,10027198,8,1,1,0.01,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2019-11-03 16:50:00,NaN,NaN,2.0,1.0,1.0,0.0,20143.0,226234.0,4A2F95BF-2AD9-4161-B6CE-B25E58133A94,2019-11-03 16:49:57,NaN,NaN
1,96979655,10032583,13,1,1,0.02,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2019-11-05 00:19:47,NaN,NaN,2.0,1.0,1.0,0.0,20144.0,226023.0,862C3D65-0285-44FD-A471-9F05690DA9AE,2019-11-05 00:17:55,NaN,NaN
2,96988675,10034191,2,1,1,0.00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2019-11-05 10:29:20,NaN,NaN,2.0,1.0,1.0,0.0,20144.0,226317.0,47B8955B-0BFD-45D7-BDD3-5AD8BDDAC5AE,2019-11-05 10:26:21,NaN,NaN
3,97008499,10037673,2,1,1,0.00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2019-11-05 16:43:50,NaN,NaN,2.0,1.0,1.0,0.0,20144.0,226277.0,B5E3068C-94FE-459C-82F7-116AEC954A05,2019-11-05 16:43:46,NaN,NaN
4,97017737,10040024,2,1,2,0.00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2019-11-05 20:16:50,NaN,NaN,2.0,1.0,1.0,0.0,20757.0,226414.0,FABFE3B8-1B2C-45F5-BAA5-B82F62FE6628,2019-11-05 19:53:01,NaN,NaN


In [9]:
data_3d.drop(['is_pay', 'channel_id', 'source_id', 'device_no', 'reg_date', 'create_role_time', 'is_gunfu'],axis=1, inplace=True)

In [10]:
data_3d.isna().sum()

user_id                 0
role_id                 0
r1                      0
r2                      0
r3                      0
r4                      0
max_role_level          0
p1                      0
p2                      0
p3                      0
p4                      0
p5                      0
p6                      0
is_create_role     211030
is_create_no       211030
server_sum         727539
platform           455633
user_type          455633
user_flag          455633
mobile             455633
game_pay_num       954045
chmoney_pay_num    954045
dtype: int64

In [11]:
data_3d.dropna(axis=0, how='any', inplace=True)
data_3d.shape

(5164431, 22)

In [12]:
day_30_product = pd.read_csv('./day_30_product.csv')

In [13]:
# 每个角色注册前30天付费总额 30d_pay_money
groups = day_30_product.groupby('role_id')
temp = groups['money'].agg([('30d_pay_money', 'sum')]).reset_index()
day_30_product = pd.merge(day_30_product, temp, on='role_id', how='left', validate='many_to_one')

In [14]:
temp = day_30_product[['role_id', '30d_pay_money']].drop_duplicates()
data = pd.merge(data_3d, temp, on='role_id', how='left', validate='one_to_one')
data.shape

(5164431, 23)

In [15]:
data.head()

,user_id,role_id,r1,r2,r3,r4,max_role_level,p1,p2,p3,p4,p5,p6,is_create_role,is_create_no,server_sum,platform,user_type,user_flag,mobile,game_pay_num,chmoney_pay_num,30d_pay_money
0,102624267,11263507,1,1,1,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,NaN
1,102630946,11265111,3,1,1,0.02,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,NaN
2,102632190,11265397,4,1,1,0.01,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,NaN
3,102645564,11269078,4,1,1,0.01,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,NaN
4,102652393,11271222,12,1,1,0.02,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,NaN


In [18]:
data[(data.p2>20000) & (data.max_role_level<5)]

,user_id,role_id,r1,r2,r3,r4,max_role_level,p1,p2,p3,p4,p5,p6,is_create_role,is_create_no,server_sum,platform,user_type,user_flag,mobile,game_pay_num,chmoney_pay_num,30d_pay_money
864878,114397724,13552789,111,2,3,0.65,1,76.0,22012.0,0.0,0.0,76.0,22012.0,0.0,0.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,40400.0
1257916,106682701,12101440,116,1,15,0.63,1,66.0,20204.0,0.0,0.0,66.0,20204.0,0.0,0.0,12.0,2.0,1.0,1.0,0.0,0.0,0.0,72350.0
2245097,95961279,136776599,27,1,3,2.65,0,140.0,54324.0,0.0,0.0,140.0,54324.0,1.0,0.0,2.0,1.0,18.0,1.0,0.0,89.0,0.0,97666.0
3863023,96051328,136866447,20,1,3,3.43,0,100.0,30298.0,0.0,0.0,100.0,30298.0,0.0,0.0,2.0,1.0,18.0,1.0,0.0,97.0,0.0,69970.0
4703242,102928657,11332082,68,1,3,1.06,1,61.0,21350.0,0.0,0.0,61.0,21350.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,11.0,0.0,43074.0
5133641,105118998,11775192,98,1,6,0.74,1,87.0,29990.0,0.0,0.0,87.0,29990.0,1.0,0.0,2.0,2.0,1.0,1.0,0.0,20.0,0.0,34150.0


In [20]:
data[data.max_role_level>250]

,user_id,role_id,r1,r2,r3,r4,max_role_level,p1,p2,p3,p4,p5,p6,is_create_role,is_create_no,server_sum,platform,user_type,user_flag,mobile,game_pay_num,chmoney_pay_num,30d_pay_money
208233,93904522,148685532,63,1,4,1.13,257,3.0,158.0,0.0,0.0,3.0,158.0,1.0,0.0,3.0,1.0,18.0,1.0,0.0,0.0,0.0,226.0
520867,96576657,151884636,14,1,9,4.96,256,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,1.0,18.0,2.0,0.0,0.0,0.0,NaN
832827,95936789,151350655,2,1,1,10.05,260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.0,1.0,18.0,1.0,0.0,0.0,0.0,NaN
1078510,90028071,151768268,5,1,2,0.81,285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,NaN
1562726,92119051,148088560,1,1,1,0.00,260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,18.0,1.0,0.0,16.0,0.0,NaN
3599910,95985857,150181798,36,1,8,1.99,267,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,1.0,0.0,5.0,0.0,128.0
4394176,92873848,149825882,27,1,5,2.63,265,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,18.0,1.0,0.0,2.0,0.0,98.0
4533614,94058657,152194013,2,1,1,0.21,254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN
4799496,96529284,150668969,22,2,10,2.87,261,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,1.0,0.0,2.0,0.0,NaN
4968683,92079971,150208567,26,1,6,2.78,275,9.0,1746.0,0.0,0.0,9.0,1581.0,1.0,1.0,3.0,1.0,18.0,1.0,0.0,0.0,0.0,3110.0


In [21]:
data.drop(['user_id'], axis=1, inplace=True)
data.fillna(0, inplace=True)
data.to_csv('./FE_data/data.csv', index=None)
del data,data_3d, day_30_product
gc.collect()

18219